In [1]:
# pip install sb3-contrib
import gym
from gym import spaces
import torch
import numpy as np

from stable_baselines3.common.vec_env import DummyVecEnv





In [9]:

#discrete_action_max = 25
# class CustomEnv(gym.Env):
#     def __init__(self, S, A, s, R):
#         self.S = torch.tensor(S, dtype=torch.float32)  # Convert S to torch tensor
#         self.A = torch.tensor(A, dtype=torch.float32)  # Convert A to torch tensor
#         self.s = s
#         if self.s.all != None:
#             self.s = torch.tensor(s, dtype=torch.float32)  # Convert Y to torch tensor
#             obs_dim_s = self.s.shape[1]
#         self.R = torch.tensor(R, dtype=torch.float32) if R is not None else None  # Convert R to torch tensor if provided
#         self.num_samples = self.S.shape[0]
#         self.current_step = 0
        
#         # Action space: assume discrete actions for simplicity
#         action_dim = self.A.shape[1]
#         #self.action_space = spaces.MultiDiscrete([self.discrete_action_max]*action_dim)
#         self.action_space = spaces.Box(low=-1e6, high=1e6, shape=(action_dim,), dtype=np.float32)
#         # Observation space: separate S and Y with specific labels
#         obs_dim_S = self.S.shape[1]
#         observation_space = {}
#         for i in range(obs_dim_S):
#             observation_space[f'S_{i+1}'] = spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32)

#         if self.s.all != None:
#             for j in range(obs_dim_s):
#                 observation_space[f's_{j+1}'] = spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32)
#         self.observation_space = spaces.Dict(observation_space)
        
#     def reset(self):
#         self.current_step = 0
#         return self._get_observation()
    
#     def step(self, action):
#         # Apply action and get next state and reward
#         next_state = self._get_observation()
#         reward = self._calculate_reward(action)
#         done = (self.current_step == self.num_samples - 1)
#         self.current_step += 1
#         return next_state, reward, done, {}
    
#     def _get_observation(self):
#         obs = {}
#         for i in range(self.S.shape[1]):
#             obs[f'S_{i+1}'] = np.array([self.S[self.current_step,i]])
#         if self.s.all != None:
#             for j in range(self.s.shape[1]):
#                 obs[f's_{j+1}'] = np.array([self.s[self.current_step,j]])
#         return obs
    
#     def _calculate_reward(self, action):
#         # Example: using a reward matrix R based on expected Y values
#         if self.R is not None:
#             ideal_y = self.R[action]  # Ideal Y value for the chosen action
#             actual_y = self.S[self.current_step]  # Actual Y value
#             reward = -torch.norm(actual_y - ideal_y)  # Negative L2 norm difference as reward
#         else:
#             # Default reward function (replace with your own logic)
#             reward = torch.sum(self.A[self.current_step] * action)
        
#         return reward

# class CustomEnv(gym.Env):
#     def __init__(self, S, A, Y, R):
#         self.S = torch.tensor(S, dtype=torch.float32)  # Convert S to torch tensor
#         self.A = torch.tensor(A, dtype=torch.float32)  # Convert A to torch tensor
#         self.Y = torch.tensor(Y, dtype=torch.float32)  # Convert Y to torch tensor
#         self.R = torch.tensor(R, dtype=torch.float32) if R is not None else None  # Convert R to torch tensor if provided
#         self.num_samples = S.shape[0]
#         self.current_step = 0
#         #self.discrete_action_max = discrete_action_max
        
#         # Action space: assume discrete actions for simplicity
#         action_dim = A.shape[1]
#         #self.action_space = spaces.MultiDiscrete([self.discrete_action_max]*action_dim)
#         self.action_space = spaces.Box(low=-1e6, high=1e6, shape=(action_dim,), dtype=np.float32)
        
#         #self.action_space = spaces.Box([self.discrete_action_max]*action_dim)
#         # Observation space: separate S and Y with specific labels
#         obs_dim_s = S.shape[1]
#         obs_dim_y = Y.shape[1]
#         observation_space = {}
#         for i in range(obs_dim_s):
#             observation_space[f'S_{i+1}'] = spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32)
#         for j in range(obs_dim_y):
#             observation_space[f'Y_{j+1}'] = spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32)
#         self.observation_space = spaces.Dict(observation_space)
        
#     def reset(self):
#         self.current_step = 0
#         return self._get_observation()
    
#     def step(self, action):
#         # Apply action and get next state and reward
#         next_state = self._get_observation()
#         reward = self._calculate_reward(action)
#         done = (self.current_step == self.num_samples - 1)
#         self.current_step += 1
#         return next_state, reward, done, {}
    
#     def _get_observation(self):
#         obs = {}
#         for i in range(self.S.shape[1]):
#             obs[f'S_{i+1}'] = np.array([self.S[self.current_step, i]])
#         for j in range(self.Y.shape[1]):
#             obs[f'Y_{j+1}'] = np.array([self.Y[self.current_step, j]])
#         return obs
    
#     def _calculate_reward(self, action):
#         # Example: using a reward matrix R based on expected Y values
#         if self.R is not None:
#             ideal_y = self.R[action]  # Ideal Y value for the chosen action
#             actual_y = self.Y[self.current_step]  # Actual Y value
#             reward = -torch.norm(actual_y - ideal_y)  # Negative L2 norm difference as reward
#         else:
#             # Default reward function (replace with your own logic)
#             reward = torch.sum(self.A[self.current_step] * action)
        
#         return reward

In [56]:
class CustomEnv(gym.Env):
    def __init__(self, S, A, R):
        self.S = S#torch.tensor(S, dtype=torch.float32)  # Convert S to torch tensor
        self.A = A#torch.tensor(A, dtype=torch.float32)  # Convert A to torch tensor
        self.R = R#torch.tensor(R, dtype=torch.float32) if R is not None else None  # Convert R to torch tensor if provided
        self.num_samples = self.S.shape[0]
        self.current_step = 0
        
        # Action space: assume discrete actions for simplicity
        self.action_dim = self.A.shape[1]
        self.action_space = spaces.Box(low=-1e6, high=1e6, shape=(self.action_dim,))
        
        # Observation space: separate S and Y with specific labels
        self.obs_dim_S = self.S.shape[1]
        # observation_space = {}
        # for i in range(obs_dim_S):
        #     observation_space[f'S_{i+1}'] = spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32)
        
        # self.observation_space = spaces.Dict(observation_space)
        self.observation_space = spaces.Box(low=-torch.inf, high=torch.inf, shape=(self.obs_dim_S,))
        #print(self.observation_space)

        
    def reset(self):
        self.current_step = 0
        return self._get_observation()
    
    def step(self, action):
        # Apply action and get next state and reward
        next_state = self._get_observation()
        action = torch.from_numpy(action.astype(np.float64) )
        reward = self._calculate_reward(action)
        #print("next_state",next_state.shape)
        #print("action",action.shape)
        #print("reward",reward.shape)
        #print("self.current_step",self.current_step)
        done = (self.current_step == self.obs_dim_S - 1)
        #print("done",done)
        self.current_step = self.current_step + 1
        #print("self.current_step",self.current_step)
        return next_state, reward, done, {}



    def _get_observation(self):
        # obs = {}
        # for i in range(self.S.shape[1]):
        #     obs[f'S_{i+1}'] = np.array([self.S[self.current_step, i]])
        obs = self.S[self.current_step, :]
        #print("obs.shape",obs.shape)
        return obs
    
    def _calculate_reward(self, action):
        # Example: using a reward matrix R based on expected Y values
        if self.R is not None:
            # 将 A 转换成 124 x 1 的矩阵以便与 SA 进行距离计算
            #print(action.shape)
            #print(self.R.shape)
            # 現在這只是單純計算 action 離 RWARD 的值哪個比較接近
            # 理論上? RWARD 的 action 維度要離散化，多少區間內給多少分數
            # 最後把該動作的rewar轉成action的矩陣
            # 但是別人怎麼做呢?
            #要看PAPER
            distances = torch.cdist(action.reshape(-1,1), self.R[:,self.current_step].reshape(-1,1), p=2)

            # Step 3: 找到最小距离对应的索引
            # np.argmin(distances, axis=1) 会返回每行中最小值的索引
            nearest_indices = torch.argmin(distances, axis=1)
            # Step 4: 根据索引找到 SA 中对应的值
            # 使用这些索引从 SA 中提取对应的值
            reward = self.R[nearest_indices,self.current_step].mean() # 整體動作的REWARD，但要怎麼取? MAX ? MEIDAN ? AVERAGE ?
            #print(reward.shape)

        else:
            #print(self.A.shape,action.shape)
            # Default reward function (replace with your own logic)
            # 這邊有問題
            reward = torch.sum(self.A[self.current_step] * action)
        
        return reward

In [60]:
# Example usage

# Example data (replace with your actual data)
state_dim = 29
action_dim = 124
S = np.random.randn(15000, state_dim)
S = torch.from_numpy(S)
A = np.random.randn(5000, action_dim)
#A = np.random.randint(low=1, high=discrete_action_max, size=(1500, 45))
A = torch.from_numpy(A)
# Example reward matrix R based on Y (replace with your own logic)
R = np.random.uniform(low=-100,high=100,size=(action_dim, state_dim))  # Example: random R matrix for 45 actions and 23 Y features
R = torch.from_numpy(R)
# Create the custom environment
env = CustomEnv(S=S, A=A, R=R)

# Wrap the environment in a vectorized form
vec_env = DummyVecEnv([lambda: env])




In [ ]:
# from stable_baselines3.common.evaluation import evaluate_policy


In [15]:
# from stable_baselines3 import PPO,A2C
# model = A2C("MultiInputPolicy", vec_env, verbose=1)
# pip install sb3-contrib

In [17]:
new_obs = dict()
for i in range(S.shape[1]):
    new_obs[f'S_{i+1}'] = np.array([np.random.randn(1)])

array([ 4,  2,  7, 15,  6, 11, 17,  4, 19])

In [140]:
# import numpy as np
# from scipy.spatial.distance import cdist

# # Step 1: 生成随机数据
# np.random.seed(42)  # 设置随机种子以保证结果可重复
# step = 2
# A = np.random.randint(1,20, (9,3)).astype(np.float64)  # SA 是 124 x 1 的矩阵
# action = np.random.randint(1,30,9).astype(np.float64)    # A 是 124 的一维数组



# A = torch.from_numpy(A)
# action = torch.from_numpy(action)

# # Step 2: 计算 A 和 SA 之间的距离
# # 将 A 转换成 124 x 1 的矩阵以便与 SA 进行距离计算
# distances = torch.cdist(action.reshape(-1,1), A[:,step].reshape(-1,1), p=1)

# # Step 3: 找到最小距离对应的索引
# # np.argmin(distances, axis=1) 会返回每行中最小值的索引
# nearest_indices = np.argmin(distances, axis=1)

# # Step 4: 根据索引找到 SA 中对应的值
# # 使用这些索引从 SA 中提取对应的值
# nearest_values = A[nearest_indices,step]

# # 打印结果
# print(nearest_values.flatten())  # 展平数组以便更清晰地查看结果



tensor([19.,  5., 19., 11.,  5., 19.,  4., 19., 19.], dtype=torch.float64)


In [45]:
action

NameError: name 'action' is not defined

In [53]:
R[:,1].mean()

tensor(2.0288, dtype=torch.float64)

In [115]:
nearest_values

tensor([19.,  5., 19., 11.,  5., 19.,  4., 19., 19.])

In [40]:
R[1,2]

56.080909518537936

In [34]:
vec_env

In [31]:
SA = np.random.randn(124, 1)
A = np.random.randn(124,)
我想透過index，依照A的值找出，SA的最接近的值，並且回傳該值。假設 

A的index 是1，他最接近SA 的index是9，其值是6，
A的index 是2，他最接近SA 的index是9，其值是6，
A的index 是3，他最接近SA 的index是2，其值是8，
那我就依照A的格式，回傳[6,6,8,.....]

SyntaxError: invalid character '，' (U+FF0C) (3618413405.py, line 3)

In [229]:
from sb3_contrib import ARS # 機率輸出不會改變
from stable_baselines3 import A2C # 機率輸出會改變
from stable_baselines3 import DDPG # 輸出機率和動作長依樣?
from stable_baselines3 import HER # 這要搭配其他使用
from stable_baselines3 import PPO # 機率輸出會改變
from stable_baselines3 import SAC # 機率輸出會改變
from stable_baselines3 import TD3 # 輸出機率和動作長依樣?
from sb3_contrib import TRPO
from sb3_contrib import TQC
from sb3_contrib import RecurrentPPO # MlpLstmPolicy


"""
when you add deterministic=True, all the predicted actions will be always determined by the maximum probability, instead of the probability by itself.

Just to give you an example, let's suppose you have the following probabilities:

25% of action A
75% of action B
If you don't use the deterministic=True, the model will use those probabilities to return a prediction.
If you use deterministic=True, the model is going to return always action B.

"""
for RL_model in [TRPO]:#ARS,A2C,DDPG,PPO,SAC,TD3,TRPO,TQC
    # Example data (replace with your actual data)
    state_dim = 29
    action_dim = 124
    S = np.random.randn(15000, state_dim)

    A = np.random.randn(5000, action_dim)
    #A = np.random.randint(low=1, high=discrete_action_max, size=(1500, 45))

    # Example reward matrix R based on Y (replace with your own logic)
    R = np.random.uniform(low=-100,high=100,size=(action_dim, state_dim))  # Example: random R matrix for 45 actions and 23 Y features

    S = torch.from_numpy(S)
    A = torch.from_numpy(A)
    R = torch.from_numpy(R)
    # Create the custom environment
    env = CustomEnv(S=S, A=A, R=R)

    # Wrap the environment in a vectorized form
    vec_env = DummyVecEnv([lambda: env])

    print(RL_model)
    # Initialize and train the RL agent (using PPO as an example)
    model = RL_model("MlpPolicy", env, verbose=1) # MultiInputPolicy
    model.learn(total_timesteps=10000)
    # new_obs = dict()
    # for i in range(S.shape[1]):
    #     new_obs[f'S_{i+1}'] = np.array([np.random.randn(1)])
    new_obs = np.random.randn(1,S.shape[1])
    new_obs = torch.from_numpy(new_obs)
    print(new_obs.shape[1])
    action, _ = model.predict(new_obs,deterministic=True) # deterministic=False輸出action的機率，true則輸出action的值
    print(action.shape)

    # r = None
    # best_action = None
    # for i in range(100):  # Test for 10 steps
    #     action, _ = model.predict(new_obs)
    #     obs, reward, done, info = env.step(action)
    #     if r == None:
    #         r = reward
    #     elif r < reward:
    #         r = reward
    #         best_action = action

    #         print(f"Action: {best_action}, Reward: {r}, {i}")
# Save and reload the model (optional)
#model.save("rl_model")
# model = PPO.load("rl_model")

# Test the trained model


# # 官網標準寫法
# env = gym.make("CartPole-v1", render_mode="human")

# model = PPO("MlpPolicy", env, verbose=1)
# model.learn(total_timesteps=10_000)

# vec_env = model.get_env()
# obs = vec_env.reset()
# for i in range(1000):
#     action, _states = model.predict(obs, deterministic=True)
#     obs, reward, done, info = vec_env.step(action)
#     vec_env.render()
#     # VecEnv resets automatically
#     # if done:
#     #   obs = env.reset()

# env.close()
# #


# # 這邊是觀察內部
# for i_episode in range(5): #how many episodes you want to run
#     observation = env.reset() #reset() returns initial observation
#     for t in range(200):
#         env.render()
#         print(observation)
#         action = env.action_space.sample()
#         observation, reward, done, info = env.step(action)
#         if done:
#             print("Episode finished after {} timesteps".format(t+1))
#             break

# # 這邊則是進行預測
# for _ in range(10):  # Test for 10 steps
#     action, _ = model.predict(obs, deterministic=True)
#     obs, reward, done, info = env.step(action)
#     print(f"Action: {action}, Reward: {reward}")

<class 'sb3_contrib.trpo.trpo.TRPO'>
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 29       |
|    ep_rew_mean     | -5.17    |
| time/              |          |
|    fps             | 1050     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                  |          |
|    ep_len_mean            | 29       |
|    ep_rew_mean            | -4.99    |
| time/                     |          |
|    fps                    | 920      |
|    iterations             | 2        |
|    time_elapsed           | 4        |
|    total_timesteps        | 4096     |
| train/                    |          |
|    explained_variance     | -0.449   |
|    is_line_search_success | 1        |
|    kl_divergence_loss     | 0.00099

In [234]:
new_obs

tensor([[-0.7871,  0.9039,  1.5504, -0.4691, -0.0908, -0.7343, -1.6763, -0.6661,
          1.0916,  0.9902,  1.9747,  0.9964,  1.8461, -1.0884, -1.3171,  1.7140,
          0.9146, -0.2111, -0.5123, -0.2285,  0.0553,  0.4445,  0.0920,  0.0404,
         -0.8069,  0.6070, -0.1988, -0.2639, -0.6756]], dtype=torch.float64)

In [256]:
action, _ = model.predict(new_obs,deterministic=True)
model.predict(new_obs,deterministic=True)

(array([[ 0.10429512,  0.05155674, -0.0768303 ,  0.01281125,  0.08048081,
          0.06557958,  0.09027342,  0.08771456,  0.05187207,  0.19846253,
          0.0664503 ,  0.03900621,  0.09570971,  0.03734688,  0.06144251,
          0.07476458,  0.07706401,  0.08076768,  0.1373069 ,  0.06146199,
          0.12250422,  0.01001924, -0.02432631,  0.11008753,  0.0784854 ,
          0.0826609 ,  0.05172022,  0.07447641,  0.08007077,  0.06980799,
          0.02578136, -0.05455348, -0.02192363, -0.01564816,  0.0992328 ,
         -0.03860246,  0.00382761,  0.12530567, -0.02701663, -0.07405656,
          0.12358637,  0.01014275, -0.02857167, -0.08359521,  0.02672946,
          0.02346602,  0.11189777,  0.028679  , -0.00456877,  0.0316258 ,
          0.03485448,  0.09060949,  0.10247485,  0.05347365,  0.04779212,
          0.10448733,  0.14473774,  0.05419446,  0.10553341,  0.1627338 ,
          0.07155152,  0.02600072,  0.14005902,  0.07336207,  0.07702881,
          0.06494298,  0.11994858,  0.

In [251]:
action, _ = model.predict(new_obs,deterministic=False)
action

array([[ 0.44271225, -0.25945538, -0.6767864 , -1.3456639 ,  0.4583108 ,
        -0.8694251 ,  0.5728355 ,  0.250616  , -1.7483984 ,  0.71500456,
         1.289273  ,  0.1096137 , -0.17016464, -0.14660917,  0.2514745 ,
         1.5701727 , -0.06559735,  0.405105  , -0.40092352,  0.1497918 ,
        -0.1611617 , -0.03163506,  0.8675919 , -0.31376326, -1.0028843 ,
         0.8298509 ,  0.6134912 ,  0.97067845,  0.25776014,  0.31684613,
        -0.6789923 , -3.0055926 , -0.02818955, -0.10384521, -1.6048154 ,
         0.55964726, -1.0570384 ,  0.92560476, -0.30416143,  0.18905807,
        -0.8934595 , -0.40526015,  1.111753  , -0.09417639,  1.4580442 ,
         0.886101  , -1.0348625 ,  0.14082423,  1.2454921 ,  0.68530387,
         1.2053729 ,  0.7213781 ,  0.71994   , -1.1614175 ,  1.0004908 ,
         0.5783862 ,  0.06592529,  0.11703116,  0.54999053, -0.77706444,
        -0.35461938,  0.8117135 ,  1.1627471 ,  0.5165836 ,  1.770658  ,
        -0.82393616,  0.80179393,  0.9695358 ,  0.6

In [287]:
best_rewards = float('-inf')
best_action = None


while True:
    action, _states = model.predict(new_obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    if not dones:  # 使用適當的布林條件檢查
        if rewards > best_rewards:
            best_rewards = rewards
            best_action = action
    else:
        break


In [288]:
best_action

array([[ 0.10429512,  0.05155674, -0.0768303 ,  0.01281125,  0.08048081,
         0.06557958,  0.09027342,  0.08771456,  0.05187207,  0.19846253,
         0.0664503 ,  0.03900621,  0.09570971,  0.03734688,  0.06144251,
         0.07476458,  0.07706401,  0.08076768,  0.1373069 ,  0.06146199,
         0.12250422,  0.01001924, -0.02432631,  0.11008753,  0.0784854 ,
         0.0826609 ,  0.05172022,  0.07447641,  0.08007077,  0.06980799,
         0.02578136, -0.05455348, -0.02192363, -0.01564816,  0.0992328 ,
        -0.03860246,  0.00382761,  0.12530567, -0.02701663, -0.07405656,
         0.12358637,  0.01014275, -0.02857167, -0.08359521,  0.02672946,
         0.02346602,  0.11189777,  0.028679  , -0.00456877,  0.0316258 ,
         0.03485448,  0.09060949,  0.10247485,  0.05347365,  0.04779212,
         0.10448733,  0.14473774,  0.05419446,  0.10553341,  0.1627338 ,
         0.07155152,  0.02600072,  0.14005902,  0.07336207,  0.07702881,
         0.06494298,  0.11994858,  0.06660507,  0.2

In [136]:
print(RecurrentPPO) # 
# Initialize and train the RL agent (using PPO as an example)
model = RecurrentPPO("MlpLstmPolicy", env, verbose=1) # MultiInputPolicy
model.learn(total_timesteps=10000)
# new_obs = dict()
# for i in range(S.shape[1]):
#     new_obs[f'S_{i+1}'] = np.array([np.random.randn(1)])
new_obs = np.random.randn(1,S.shape[1])
new_obs = torch.from_numpy(new_obs)
print(new_obs.shape[1])
action, _ = model.predict(new_obs,deterministic=True)
print(action.shape)

<class 'sb3_contrib.ppo_recurrent.ppo_recurrent.RecurrentPPO'>
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 29       |
|    ep_rew_mean     | 0.692    |
| time/              |          |
|    fps             | 410      |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 128      |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 29          |
|    ep_rew_mean          | 0.611       |
| time/                   |             |
|    fps                  | 174         |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 256         |
| train/                  |             |
|    approx_kl            | 0.040017627 |
|    clip_fraction        | 0.261       |
|    clip_range           | 0.2         |
|    entropy_loss         | -176        |
|    explained_variance   | -0.111      |
|    learning_rate        | 0.

In [164]:
action, _ = model.predict(new_obs,deterministic=True)
action

array([[ 1.64193556e-01, -1.30733192e-01,  1.69350788e-01,
         1.03534460e-01,  3.47802222e-01,  3.11723407e-02,
         3.89672905e-01,  2.71234572e-01,  2.34658375e-01,
         2.57128745e-01,  2.93758839e-01,  1.74956918e-01,
         2.02014759e-01,  1.64778739e-01,  1.39585614e-01,
         1.68799058e-01,  1.76782727e-01,  2.54211605e-01,
         9.34491456e-02,  4.07167137e-01,  7.59055242e-02,
         1.25501037e-01,  1.10017411e-01,  1.11715980e-01,
         2.76867092e-01,  2.01592118e-01,  1.05613992e-01,
         2.06894234e-01,  4.40128595e-02,  3.04735243e-01,
         3.06183755e-01,  1.49329588e-01,  1.02381542e-01,
         1.18145354e-01,  1.44413441e-01, -1.48292914e-01,
         3.23862582e-01,  2.95034111e-01,  2.57538795e-01,
         4.37002955e-03, -9.48045179e-02, -5.39447106e-02,
         1.57504901e-01,  1.21384166e-01,  3.82774442e-01,
         9.79488343e-02,  2.69628525e-01,  1.42206281e-01,
         1.49647385e-01,  3.87357295e-01,  1.27654716e-0

In [3]:
state_dim = 29
action_dim = 124
S = np.random.randn(15000, state_dim)

A = np.random.randn(5000, action_dim)
#A = np.random.randint(low=1, high=discrete_action_max, size=(1500, 45))

# Example reward matrix R based on Y (replace with your own logic)
R = np.random.uniform(low=-100,high=100,size=(action_dim, state_dim))  # Example: random R matrix for 45 actions and 23 Y features

# Create the custom environment
env = CustomEnv(S=S, A=A, R=None)

# Wrap the environment in a vectorized form
vec_env = DummyVecEnv([lambda: env])

NameError: name 'np' is not defined

In [2]:
model = RL_model("MultiInputPolicy", vec_env, verbose=1)

NameError: name 'vec_env' is not defined

In [28]:
# ARS,DDPG,HER,SAC,TD3,TQC,RecurrentPPO

In [50]:
RL_model = DDPG
# from stable_baselines3 import HerReplayBuffer
# model = SAC(
#     "MultiInputPolicy",
#     env,
#     replay_buffer_class=HerReplayBuffer,
#     replay_buffer_kwargs=dict(
#       n_sampled_goal=n_sampled_goal,
#       goal_selection_strategy="future",
#     ),
#     verbose=1,
#     buffer_size=int(1e6),
#     learning_rate=1e-3,
#     gamma=0.95,
#     batch_size=256,
#     policy_kwargs=dict(net_arch=[256, 256, 256]),
# )

In [52]:
state_dim = 29
action_dim = 124
S = np.random.randn(15000, state_dim)

A = np.random.randn(5000, action_dim)
#A = np.random.randint(low=1, high=discrete_action_max, size=(1500, 45))

# Example reward matrix R based on Y (replace with your own logic)
R = np.random.uniform(low=-100,high=100,size=(action_dim, state_dim))  # Example: random R matrix for 45 actions and 23 Y features

# Create the custom environment
env = CustomEnv(S=S, A=A, R=R)

# Wrap the environment in a vectorized form
vec_env = DummyVecEnv([lambda: env])

print(RL_model)
# Initialize and train the RL agent (using PPO as an example)
model = RL_model("MultiInputPolicy", vec_env, verbose=1)
model.learn(total_timesteps=10000)

<class 'stable_baselines3.ddpg.ddpg.DDPG'>
Using cpu device


c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


IndexError: index -450110 is out of bounds for dimension 0 with size 124

In [ ]:
#DDPG
#
# ...
# ---> 42         ideal_y = self.R[action]  # Ideal Y value for the chosen action
#      43         actual_y = self.S[self.current_step]  # Actual Y value
#      44         reward = -torch.norm(actual_y - ideal_y)  # Negative L2 norm difference as reward

# IndexError: index -370166 is out of bounds for dimension 0 with size 124


# ---> 42         ideal_y = self.R[action]  # Ideal Y value for the chosen action
#      43         actual_y = self.S[self.current_step]  # Actual Y value
#      44         reward = -torch.norm(actual_y - ideal_y)  # Negative L2 norm difference as reward

# IndexError: index -386851 is out of bounds for dimension 0 with size 124


# TD3
---> 42         ideal_y = self.R[action]  # Ideal Y value for the chosen action
     43         actual_y = self.S[self.current_step]  # Actual Y value
     44         reward = -torch.norm(actual_y - ideal_y)  # Negative L2 norm difference as reward

IndexError: index -736909 is out of bounds for dimension 0 with size 124
     

# TQC
---> 42         ideal_y = self.R[action]  # Ideal Y value for the chosen action
     43         actual_y = self.S[self.current_step]  # Actual Y value
     44         reward = -torch.norm(actual_y - ideal_y)  # Negative L2 norm difference as reward

IndexError: index -550305 is out of bounds for dimension 0 with size 124